In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_admission=pd.read_csv('ADMISSIONS.csv')
df_patient=pd.read_csv('PATIENTS.csv')
df_notes=pd.read_csv('NOTEEVENTS.csv')

# Part 1: Preprocess df_admission

In [ ]:
## Merge df_admission and df_patient
df_admission=df_admission.merge(df_patient.iloc[:,1:4], on='SUBJECT_ID', how='left')
df_admission.head()

In [ ]:
# Convert dates to Datetime for the table
df_admission.ADMITTIME = pd.to_datetime(df_admission.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_admission.DISCHTIME = pd.to_datetime(df_admission.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_admission.DEATHTIME = pd.to_datetime(df_admission.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_admission.DOB=pd.to_datetime(df_admission.DOB, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [ ]:
df_admission['AGE_UPON_ADMIT']=''
for i in range(len(df_admission)):
    df_admission['AGE_UPON_ADMIT'][i]=round((df_admission.ADMITTIME[i].to_pydatetime()-df_admission.DOB[i].to_pydatetime()).days/365,0)
df_admission.head()

In [ ]:
# Only keep patients between 18-90 and reset the index
df_admission=df_admission[(df_admission.AGE_UPON_ADMIT>=18)&(df_admission.AGE_UPON_ADMIT<=90)]
df_admission = df_admission.reset_index(drop = True)
df_admission.head()

In [ ]:
# sort by subject_ID and admission date
df_admission = df_admission.sort_values(['SUBJECT_ID','ADMITTIME'])
df_admission = df_admission.reset_index(drop = True)

In [ ]:
# Create READMISSION_TIME and READMISSION_TYPE. For admissions with "ELECTIVE" as the next readmission, use backfill to replace the na data
df_admission['READMISSION_TIME']=np.nan
df_admission['READMISSION_TYPE']=np.nan
for i in range(len(df_admission)-1):
    if df_admission['SUBJECT_ID'][i]==df_admission['SUBJECT_ID'][i+1] and df_admission['ADMISSION_TYPE'][i+1]!='ELECTIVE':
        df_admission['READMISSION_TIME'][i]=df_admission['ADMITTIME'][i+1]
        df_admission['READMISSION_TYPE'][i]=df_admission['ADMISSION_TYPE'][i+1]
df_admission[['READMISSION_TIME','READMISSION_TYPE']] = df_admission.groupby(['SUBJECT_ID'])[['READMISSION_TIME','READMISSION_TYPE']].fillna(method = 'bfill')
df_admission.head(20)

In [ ]:
# Calculate date interval between discharge date and the next admission date
df_admission['READMISSION_INTERVAL']=(df_admission.READMISSION_TIME-df_admission.DISCHTIME).dt.days
df_admission.head()

In [ ]:
# Plot the date interval and see if there's any outlier/error cases. Some date intervals are less than zero and need to be removed.
plt.hist(df_admission.READMISSION_INTERVAL, bins='auto') 
plt.title("Readmission interval")
plt.show()

In [ ]:
# Define readmission as a re-occurance of admission within 30 days after the last discharge
df_admission['READMISSION_STATUS'] = np.where((df_admission['READMISSION_INTERVAL']<=30) & (df_admission['READMISSION_INTERVAL']>=0) ,'Readmitted','Non-readmitted')

In [ ]:
# Outcome label distribution
df_admission.READMISSION_STATUS.value_counts()

# Part 2: Preprocess df_notes

In [ ]:
df_notes.info()

In [ ]:
df_notes.CATEGORY.value_counts()

In [ ]:
# Filter out the discharge summaries
df_dis_sum=df_notes[df_notes.CATEGORY=='Discharge summary']

In [ ]:
df_dis_sum=df_dis_sum.sort_values(['HADM_ID','CHARTDATE']).reset_index(drop = True)

In [ ]:
# Check if only one discharge summary can be matched with each admission id. 
# Turns out that 12646 of the notes records have duplicate admission ids, which means there could be multiple discharge summaries for one single admission.
duplicates=df_dis_sum[df_dis_sum.duplicated(['HADM_ID'],keep=False)]
len(duplicates)

In [ ]:
# combine discharge summaries for each admission id
df_dis_sum['TEXT_AGG'] = df_dis_sum.groupby(['HADM_ID'])['TEXT'].transform(lambda x: '|'.join(x))

In [ ]:
# Drop duplicates based on HADM_ID and keep only 'HADM_ID','TEXT_AGG' columns
df_dis_sum_agg = df_dis_sum[['HADM_ID','TEXT_AGG']].drop_duplicates(subset=['HADM_ID']).reset_index(drop = True)

# Part 3: Combine df_admission and df_dis_sum_agg

In [ ]:
# Merge the two tables
df_adm_dis_sum=df_admission.merge(df_dis_sum_agg, on='HADM_ID', how='left').reset_index(drop = True)

In [ ]:
# Check if any admission record can't be matched with a discharge summary
df_adm_dis_sum[df_adm_dis_sum['TEXT_AGG'].isnull()]

In [ ]:
# Only keep the rows in which the discharge summary is not null
df_adm_dis_sum=df_adm_dis_sum[df_adm_dis_sum['TEXT_AGG'].notnull()].reset_index(drop = True)

In [ ]:
# Check the distribution of positive and negative labels in the outcome variable
df_adm_dis_sum.READMISSION_STATUS.value_counts()

In [ ]:
# Save the combined file into df_adm_dis_sum_not_cleaned.csv
df_adm_dis_sum.to_csv(r'C:\USFCA\HS 651\df_adm_dis_sum_not_cleaned.csv')

# Part 4: Data cleaning for notes

In [ ]:
# I didn't use word frequency to determine stopwords but here's the code to identify the most common words:
# from collections import Counter
# sort_orders = sorted(Counter(word).items(), key=lambda x: x[1], reverse=True)
# for i in sort_orders:
    # print(i[0], i[1])

In [ ]:
# Set up punctuation and number list
import string
punc_numb=string.punctuation+'0123456789'

# Load in spacy
import spacy
nlp = spacy.load('en_core_web_sm')
stops=nlp.Defaults.stop_words

# Define text cleaning function
def clean_text(text):
    text=text.replace('\n',' ')
    text=text.replace('\r',' ')
    
    # Replace all the punctuations and numbers to space
    t = text.maketrans(dict.fromkeys(punc_numb,' '))
    text = text.lower().translate(t)
    
    # Lemmatize the words in text and save the words in list 'text_lemma'
    text=nlp(text)
    text_lemma=[]
    for token in text:
        text_lemma.append(token.lemma_)
    
    # Remove stop words and words with length<3 and extra spaces
    text_lemma_nonstop = [w for w in text_lemma if not w in stops and len(w) >= 3 and not str(w).isspace()]
    
    # Concatenate items in the list to string
    text_clean = " ".join(text_lemma_nonstop)
    
    return text_clean

In [ ]:
# Add progress bar to the cell below to check remaining executing time. 
# The tqdm function only works for the for loops so I didnt use the lambda function here. The process took me 5 hours.
from tqdm import tqdm

In [ ]:
for i in tqdm(range(len(df_adm_dis_sum))):
    df_adm_dis_sum['TEXT_AGG'][i]=clean_text(df_adm_dis_sum['TEXT_AGG'][i])

In [ ]:
# Another approach is to use the lambda fucntion: df_adm_dis_sum['TEXT_AGG']=df_adm_dis_sum['TEXT_AGG'].map(lambda x: clean_text(x))
# However, the progress of execution won't be shown and it's hard to debug

In [ ]:
# Save the processed dataframe to a csv file for modeling use
df_adm_dis_sum.to_csv(r'C:\USFCA\HS 651\df_adm_dis_sum.csv')